In [1]:
# Import dataset
import os
import pathlib
import shutil
import tensorflow as tf
import zipfile

# Get dataset from GitHub
data_dir = tf.keras.utils.get_file(
    "dataset.zip",
    "https://github.com/opixelum/pokedex/blob/main/dataset.zip?raw=true",
    extract=False
)

# Extract zip file
with zipfile.ZipFile(data_dir, 'r') as zip_ref:
  zip_ref.extractall('/content/datasets')

# Get number of images
data_dir = pathlib.Path('/content/datasets/dataset')
image_count = len(list(data_dir.glob('*/*')))
print(image_count, "images downloaded.")

78296543/78296543 [==============================] - 1s 0us/step
1139 images downloaded.


In [2]:
# Prepare dataset for training & validation
batch_size = 3
image_height = 200
image_width = 200

training_data = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="training",
    seed=42,
    image_size=(image_height, image_width),
    batch_size=batch_size
)

validation_data = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="validation",
    seed=42,
    image_size=(image_height, image_width),
    batch_size=batch_size
)

Found 1113 files belonging to 2 classes.
Using 891 files for training.
Found 1113 files belonging to 2 classes.
Using 222 files for validation.


In [3]:
# Define convolutional neural network
from tensorflow import keras
from keras import layers

num_classes = 2

model = tf.keras.Sequential([
    layers.experimental.preprocessing.Rescaling(1./255),
    layers.Conv2D(128, 4, activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(64, 4, activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(32, 4, activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(16, 4, activation='relu'),
    layers.MaxPooling2D(),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(num_classes, activation='softmax')
])

In [4]:
# Train
model.compile(
    optimizer='adam',
    loss=tf.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)

logdir="logs"
tensorboard_callback = keras.callbacks.TensorBoard(
    log_dir=logdir,
    histogram_freq=1,
    write_images=logdir,
    embeddings_data=training_data
)

model.fit(
    training_data,
    validation_data=validation_data,
    epochs=20,
    callbacks=[tensorboard_callback]
)

Epoch 1/20
 96/297 [========>.....................] - ETA: 4s - loss: 0.6339 - accuracy: 0.6840

InvalidArgumentError: Graph execution error:

Detected at node decode_image/DecodeImage defined at (most recent call last):
<stack traces unavailable>
2 root error(s) found.
  (0) INVALID_ARGUMENT:  Unknown image file format. One of JPEG, PNG, GIF, BMP required.
	 [[{{node decode_image/DecodeImage}}]]
	 [[IteratorGetNext]]
  (1) CANCELLED:  Function was cancelled before it was started
0 successful operations.
0 derived errors ignored. [Op:__inference_train_function_1569]

In [ ]:
# Inference
import cv2
from google.colab import files
import matplotlib.pyplot as plt
import numpy as np

file_to_predict = files.upload()

for file_ in file_to_predict:
    image_to_predict = cv2.imread(file_,cv2.IMREAD_COLOR)
    plt.imshow(cv2.cvtColor(image_to_predict, cv2.COLOR_BGR2RGB))
    plt.show()
    img_to_predict = np.expand_dims(cv2.resize(image_to_predict,(200,200)), axis=0)

    predict = model.predict(img_to_predict)
    res = np.argmax(predict, axis=1)

    if res == 0:
      print("It's an Lucario!")
    elif res == 1:
      print("It's a Entei!")